<a href="https://colab.research.google.com/github/VishanOberoi/Torch/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data  import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Create a Fully Connected Network


In [ ]:


class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self,x):
    x  = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x



# Create the CNN

In [13]:
class CNN(nn.Module):
  def __init__(self, in_channel = 1, num_classes = 10):
    super(CNN, self).__init__()
    #Same convolution
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (3,3), stride = (1,1), padding = (1,1))
    self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2)) # convert to 14x14
    self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (3,3), stride = (1,1), padding = (1,1))
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x =  F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1) #Need it to one dim
    x = self.fc1(x)

    return x

model = CNN()
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)




torch.Size([64, 10])


# Set device

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_channel = 1
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 5
device

device(type='cuda')

#Load Data

In [15]:
train_dataset = datasets.MNIST(root = 'dataset/', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size , shuffle = True)

test_dataset =  datasets.MNIST(root = 'dataset/', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size , shuffle = True)


# Initialize the network



In [16]:
model = CNN().to(device)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)


#Train the network


In [18]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device = device)
    targets = targets.to(device = device)


    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    #backward
    optimizer.zero_grad() #set all gradients to zero for each new batch
    loss.backward()

    #Adam step
    optimizer.step() #Update weights









In [19]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking acc on training data")
  else:
    print("Checking acc on testing data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad(): #Dont need to compute the gradients
    for x,y in loader:
      x = x.to(device = device)
      y = y.to(device = device)

      scores = model(x) #There are 10 digits, give the max prob
      scores.max(1) #Shape is 64x10, We want vals for 10 part
      _, preds = scores.max(1)
      num_correct +=  (preds == y).sum()
      num_samples += preds.size(0)
    #Convert the tensors to floats
    print(f"Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}")

  model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking acc on training data
Got 59141/ 60000 with accuracy 98.56833333333334
Checking acc on testing data
Got 9829/ 10000 with accuracy 98.29


## A simple CNN